# Import Comet for hyperparameter optimization

In [49]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/7b440e5ad74f4fc79c5f4cd45b9bcc6d



In [50]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

# Import libraries and data

In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input,Embedding,Flatten,Concatenate,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model
%matplotlib inline
sns.set_style('whitegrid')

In [52]:
df_train_features = pd.read_csv('data/train_features.csv')
df_train_labels = pd.read_csv('data/train_labels.csv')

df_valid_features = pd.read_csv('data/valid_features.csv')
df_valid_labels = pd.read_csv('data/valid_labels.csv')

df_test_features = pd.read_csv('data/test_features.csv')

# Preprocess data

In [53]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

In [54]:
# num_train_data['loudness'] = df_train_features['loudness']
# num_train_data['tempo'] = df_train_features['tempo']
# num_train_data['duration'] = df_train_features['duration']

# num_valid_data['loudness'] = df_valid_features['loudness']
# num_valid_data['tempo'] = df_valid_features['tempo']
# num_valid_data['duration'] = df_valid_features['duration']

# num_test_data['loudness'] = df_test_features['loudness']
# num_test_data['tempo'] = df_test_features['tempo']
# num_test_data['duration'] = df_test_features['duration']

In [55]:
cat_train_data = df_train_features[['time_signature', 'key', 'mode']]
cat_valid_data = df_valid_features[['time_signature', 'key', 'mode']]
cat_test_data = df_test_features[['time_signature', 'key', 'mode']]

In [56]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [57]:
cat_train_data['time_signature_cat'] = le.fit_transform(cat_train_data['time_signature'])
cat_train_data['key_cat'] = le.fit_transform(cat_train_data['key'])
cat_train_data['mode_cat'] = le.fit_transform(cat_train_data['mode'])

cat_valid_data['time_signature_cat'] = le.fit_transform(cat_valid_data['time_signature'])
cat_valid_data['key_cat'] = le.fit_transform(cat_valid_data['key'])
cat_valid_data['mode_cat'] = le.fit_transform(cat_valid_data['mode'])

cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
cat_test_data['mode_cat'] = le.fit_transform(cat_test_data['mode'])

C:\Users\alex0\AppData\Local\Temp/ipykernel_26628/3557455032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_train_data['time_signature_cat'] = le.fit_transform(cat_train_data['time_signature'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_26628/3557455032.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_train_data['key_cat'] = le.fit_transform(cat_train_data['key'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_26628/3557455032.py:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [58]:
cat_train_data = cat_train_data.drop(['time_signature', 'key', 'mode'], axis=1)
cat_valid_data = cat_valid_data.drop(['time_signature', 'key', 'mode'], axis=1)
cat_test_data = cat_test_data.drop(['time_signature', 'key', 'mode'], axis=1)

In [59]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [60]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)
num_test = scaler.fit_transform(num_test_data)

In [61]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

# Categorical inputs

In [62]:
# size is the number of unique values in each characteristic
# size can be other values as well since it will generate numbers of vectors
time_signature_size = 6
key_size = 12
mode_size = 2

In [63]:
time_signature_input = Input(shape=(1,), name='time_signature')
key_input = Input(shape=(1,), name='key')
mode_input = Input(shape=(1,), name='mode')

In [64]:
# the min(600, round(1.6 * key_size**0.56)) is the rule of thumb for the input size(weight)
# from  fast.ai, Jeremy Howard
# reference from : https://bit.ly/3jsGw4u
time_signature_embedded = Embedding(cat_train_data.time_signature_cat.max()+1, min(600, round(1.6 * time_signature_size**0.56)), input_length=1, name='time_signature_embedded')(time_signature_input)
key_embedded = Embedding(cat_train_data.key_cat.max()+1, min(600, round(1.6 * key_size**0.56)),input_length=1,name='key_embedded')(key_input)
mode_embedded = Embedding(cat_train_data.mode_cat.max()+1, min(600, round(1.6 * mode_size**0.56)),input_length=1,name='mode_embedded')(mode_input)

In [65]:
cat_train = [cat_train_data.time_signature_cat,cat_train_data.key_cat,cat_train_data.mode_cat]
cat_valid = [cat_valid_data.time_signature_cat,cat_valid_data.key_cat,cat_valid_data.mode_cat]

# Weight adjustment and early stopping settings

In [66]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df_train_labels['genre']),y=df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

In [67]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)

# Text model

In [68]:
import ktrain
from ktrain import text

In [69]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [70]:
# remove stopwords
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [71]:
x_train = df_train_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_train = df_train_labels['genre'].to_numpy()

x_test = df_valid_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_test = df_valid_labels['genre'].to_numpy()

In [72]:
x_train = x_train.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
x_test = x_test.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))

In [73]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [74]:
trn2, val2, preproc2 = text.texts_from_array(
    x_train=x_train, y_train=y_train,
    x_test=x_test, y_test=y_test,
    ngram_range=1, 
    maxlen=150,
    preprocess_mode='standard'
)

language: en
Word Counts: 4776
Nrows: 7678
7678 train sequences
train sequence lengths:
	mean : 52
	95percentile : 95
	99percentile : 146
x_train shape: (7678,150)
y_train shape: (7678, 8)
Is Multi-Label? False
450 test sequences
test sequence lengths:
	mean : 50
	95percentile : 101
	99percentile : 129
x_test shape: (450,150)
y_test shape: (450, 8)
task: text classification


In [75]:
model2 = text.text_classifier('nbsvm', train_data=trn2, preproc=preproc2, metrics=['accuracy'])

Is Multi-Label? False
compiling word ID features...
maxlen is 150
building document-term matrix... this may take a few moments...
rows: 1-7678
computing log-count ratios...
done.


## The Dataset class for binding the audio data and the text data

In [76]:
import math
class MyCustomDataset(ktrain.SequenceDataset):
    def __init__(self, x, y, batch_size=32, shuffle=True):
        # error checks
        err = False
        if type(x) == np.ndarray and len(x.shape) != 2: err = True
        elif type(x) == list:
            for d in x:
                if type(d) != np.ndarray or len(d.shape) != 2:
                    err = True
                    break
        else: err = True
        if err:
            raise ValueError('x must be a 2d numpy array or a list of 2d numpy arrays')
        if type(y) != np.ndarray:
            raise ValueError('y must be a numpy array')
        if type(x) == np.ndarray:
            x = [x]
        # set variables

        super().__init__(batch_size=batch_size)
        self.x, self.y = x, y
        self.indices = np.arange(self.x[0].shape[0])
        self.n_inputs = len(x)
        self.shuffle = shuffle

    # required for instances of tf.keras.utils.Sequence
    def __len__(self):
        """Number of batches in the Sequence.
        """
        return math.ceil(self.x[0].shape[0] / self.batch_size)

    # required for instances of tf.keras.utils.Sequence
    def __getitem__(self, idx):
        """Gets batch at position `idx`.
        """
        # the starting and ending indices of the batch
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = []

        for i in range(self.n_inputs):
            batch_x.append(self.x[i][inds])
        batch_y = np.array(self.y[inds])
        return tuple(batch_x) , batch_y

    # required for instances of ktrain.Dataset
    def nsamples(self):
        return self.x[0].shape[0]

    #required for instances of ktrain.Dataset
    def get_y(self):
        return self.y

    def on_epoch_end(self):
        if self.shuffle:  np.random.shuffle(self.indices)

In [77]:
time_signature_train = np.expand_dims(cat_train_data.time_signature_cat, -1)
key_train = np.expand_dims(cat_train_data.key_cat, -1)
mode_train = np.expand_dims(cat_train_data.mode_cat, -1)

time_signature_valid = np.expand_dims(cat_valid_data.time_signature_cat, -1)
key_valid = np.expand_dims(cat_valid_data.key_cat, -1)
mode_valid = np.expand_dims(cat_valid_data.mode_cat, -1)

time_signature_test = np.expand_dims(cat_test_data.time_signature_cat, -1)
key_test = np.expand_dims(cat_test_data.key_cat, -1)
mode_test = np.expand_dims(cat_test_data.mode_cat, -1)

In [78]:
train_data = MyCustomDataset([num_train] + [trn2[0]] + [time_signature_train] + [key_train] + [mode_train], trn2[1])
valid_data = MyCustomDataset([num_valid] + [val2[0]] + [time_signature_valid] + [key_valid] + [mode_valid], val2[1])

# Building model

In [79]:
class NumericGenreClassifier:
    def __init__(self,num_train,cat_train,train_data,num_train_y,num_valid,cat_valid,valid_data,num_valid_y,class_weight,early_stop,experiment,text_model):
        # numeric and label input
        self.num_train = num_train
        self.num_train_y = num_train_y
        self.num_valid = num_valid
        self.num_valid_y = num_valid_y

        # categorical input
        self.cat_train = cat_train
        self.cat_valid = cat_valid

        # customed dataset
        self.train_data = train_data
        self.valid_data = valid_data

        # text model
        self.text_model = text_model

        # class weight and early stop mechanism
        self.class_weight = class_weight
        self.early_stop = early_stop

        # comet_ml experiment
        self.experiment = experiment
    
    def build_model(self):
        # numeric input layers
        numerical_input = Input(shape=(self.num_train.shape[1],), name='numerical_input')
        x = Dense(
            self.experiment.get_parameter("num_first_layer_units"),
            activation='elu',
            kernel_initializer='HeNormal')(numerical_input)
        x = Dropout(0.5)(x)
        x = BatchNormalization()(x)

        x = Dense(
            self.experiment.get_parameter("num_second_layer_units"),
            activation='elu',
            kernel_initializer='HeNormal')(x)
        x = Dropout(0.3)(x)
        x = BatchNormalization()(x)
        numerical_output = Flatten()(x) 

        # categorical input layers
        categorical_input = [time_signature_input,key_input,mode_input]
        concatenated = Concatenate(name='category')([time_signature_embedded, key_embedded, mode_embedded])
        embedded_input = Flatten()(concatenated)
        y = Dense(
            units=self.experiment.get_parameter("cat_first_layer_units"),
            activation='elu',
            kernel_initializer='HeUniform')(embedded_input)
        y = Dropout(0.5)(y)
        y = BatchNormalization()(y)

        y = Dense(
            units=self.experiment.get_parameter("cat_second_layer_units"),
            activation='elu',
            kernel_initializer='HeUniform')(y)
        y = Dropout(0.3)(y)
        y = BatchNormalization()(y)
        cat_out = y

        # final layer and output
        final_layer = Concatenate()([numerical_output, self.text_model.output, cat_out])
        final_layer = Dense(
                        units=self.experiment.get_parameter("final_layer_units"),
                        activation='elu',
                        kernel_initializer='HeNormal')(final_layer)
       
        final_model_output = Dense(8,activation='softmax', kernel_initializer='HeNormal')(final_layer)

        model = Model(inputs=[numerical_input] + [self.text_model.input] + [categorical_input],outputs=final_model_output,name='final_output')
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    def train_model(self):
        # Train the model
        combined_model = self.build_model()
        model = ktrain.get_learner(
            combined_model,
            train_data=self.train_data,
            val_data=self.valid_data,
            batch_size=self.experiment.get_parameter("batch_size")
        )
        model.fit_onecycle(
            lr=0.001,
            epochs=self.experiment.get_parameter("epochs"),
            class_weight=self.class_weight,
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        # score = model.evaluate(self.num_valid,self.num_valid_y)
        # LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [80]:
numerical_model = NumericGenreClassifier(num_train,cat_train,train_data,num_train_y,num_valid,cat_valid,valid_data,num_valid_y,class_weight,early_stop,experiment, model2)
# mu is the mean number of units, sigma is the standard deviation
numerical_model.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "num_first_layer_units": {"type": "discrete", "values": [700,750,800,850,900,950,1000,1050,1100]},
        "num_second_layer_units": {"type": "discrete", "values": [60,75,100,150,200,250,275,300,325,350]},
        "cat_first_layer_units": {"type": "discrete", "values": [200,250,300,325,350]},
        "cat_second_layer_units": {"type": "discrete", "values": [50,75,100,125,150,200]},
        "final_layer_units": {"type": "discrete", "values": [30,35,40,45,50,60,100,150]},
        "batch_size": {"type": "discrete", "values": [8,16,32]},
        "epochs": {"type": "discrete", "values": [4,5,6,7,8,9]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=535cd280b3d441838d03243f2a434846
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 388800, 'endTime': None, 'id': '535cd280b3d441838d03243f2a434846', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [8, 16, 32]}, 'cat_first_layer_units': {'type': 'discrete', 'values': [200, 250, 300, 325, 350]}, 'cat_second_layer_units': {'type': 'discrete', 'values': [50, 75, 100, 125, 150, 200]}, 'epochs': {'type': 'discrete', 'values': [4, 5, 6, 7, 8, 9]}, 'final_layer_units': {'type': 'discrete', 'values': [30, 35, 40, 45, 50, 60, 100, 150]}, 'num_first_layer_units': {'type': 'discrete', 'values': [700, 750, 800, 850, 900, 950, 1000, 1050, 1100]}, 'num_second_layer_units': {'type': 'discrete', 'values': [60, 75, 100, 150, 200, 250, 275, 300, 325, 350]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleS



begin training using onecycle policy with max lr of 0.001...
Epoch 1/5
480/480 [==============================] - 11s 18ms/step - loss: 2.0194 - accuracy: 0.3074 - val_loss: 1.4516 - val_accuracy: 0.4422
Epoch 2/5
480/480 [==============================] - 8s 17ms/step - loss: 1.5594 - accuracy: 0.4259 - val_loss: 1.3977 - val_accuracy: 0.5044
Epoch 3/5
480/480 [==============================] - 9s 18ms/step - loss: 1.3506 - accuracy: 0.4953 - val_loss: 1.1754 - val_accuracy: 0.6400
Epoch 4/5
480/480 [==============================] - 9s 18ms/step - loss: 1.2361 - accuracy: 0.5393 - val_loss: 1.1000 - val_accuracy: 0.6422
Epoch 5/5
480/480 [==============================] - 9s 18ms/step - loss: 1.1263 - accuracy: 0.5784 - val_loss: 1.1040 - val_accuracy: 0.6378


begin training using onecycle policy with max lr of 0.001...
Epoch 1/5
480/480 [==============================] - 11s 20ms/step - loss: 1.9925 - accuracy: 0.3111 - val_loss: 1.3342 - val_accuracy: 0.5289
Epoch 2/5
480/480 [=

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/94eeab8e480f4695a6841558f549aee9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.30737170577049255, 0.5995050668716431)
COMET INFO:     batch_accuracy [480]         : (0.0, 0.625)
COMET INFO:     batch_loss [480]             : (0.7126136422157288, 3.0968096256256104)
COMET INFO:     epoch_duration [10]          : (8.360000000000582, 11.202999999994063)
COMET INFO:     loss [10]                    : (1.0653905868530273, 2.0193936824798584)
COMET INFO:     val_accuracy [10]            : (0.4422222077846527, 0.6466666460037231)
COMET INFO:     val_loss [10]                : (1.062783122062683, 1.4515583515167236)
COMET INFO:     validate_batch_accuracy [



begin training using onecycle policy with max lr of 0.001...
Epoch 1/7
960/960 [==============================] - 20s 19ms/step - loss: 2.0012 - accuracy: 0.3210 - val_loss: 1.4235 - val_accuracy: 0.4689
Epoch 2/7
960/960 [==============================] - 17s 17ms/step - loss: 1.5303 - accuracy: 0.4418 - val_loss: 1.2591 - val_accuracy: 0.5778
Epoch 3/7
960/960 [==============================] - 14s 15ms/step - loss: 1.3302 - accuracy: 0.5078 - val_loss: 1.3045 - val_accuracy: 0.5689
Epoch 4/7
960/960 [==============================] - 10s 10ms/step - loss: 1.2020 - accuracy: 0.5565 - val_loss: 1.2119 - val_accuracy: 0.6178
Epoch 5/7
960/960 [==============================] - 8s 9ms/step - loss: 1.0693 - accuracy: 0.5979 - val_loss: 1.0677 - val_accuracy: 0.6356
Epoch 6/7
960/960 [==============================] - 8s 8ms/step - loss: 0.9431 - accuracy: 0.6482 - val_loss: 1.0524 - val_accuracy: 0.6267
Epoch 7/7
960/960 [==============================] - 9s 9ms/step - loss: 0.8500 - a

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/f38667be9782471c96a8ab5bcb48123d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [14]                : (0.3210471570491791, 0.7155508995056152)
COMET INFO:     batch_accuracy [1344]        : (0.0, 0.875)
COMET INFO:     batch_loss [1344]            : (0.7466440200805664, 3.881157636642456)
COMET INFO:     epoch_duration [14]          : (7.030999999995402, 19.75)
COMET INFO:     loss [14]                    : (0.7564696073532104, 2.0227367877960205)
COMET INFO:     val_accuracy [14]            : (0.46222221851348877, 0.6622222065925598)
COMET INFO:     val_loss [14]                : (1.0175449848175049, 1.4437315464019775)
COMET INFO:     validate_batch_accuracy [28] : (0.4062

...

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/ffa8b1ed238b42569eaab39c28aeece2
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [14]                : (0.32247981429100037, 0.8095858097076416)
COMET INFO:     batch_accuracy [1344]        : (0.07954545319080353, 1.0)
COMET INFO:     batch_loss [1344]            : (0.16300787031650543, 3.2873401641845703)
COMET INFO:     epoch_duration [14]          : (7.187999999994645, 10.202999999994063)
COMET INFO:     loss [14]                    : (0.4907558560371399, 1.9186409711837769)
COMET INFO:     val_accuracy [14]            : (0.4888888895511627, 0.653333306312561)
COMET INFO:     val_loss [14]                : (1.1208034753799438, 1.4200776815414429)
COMET INFO:     validate_b



begin training using onecycle policy with max lr of 0.001...
Epoch 1/6
480/480 [==============================] - 5s 8ms/step - loss: 2.0928 - accuracy: 0.2731 - val_loss: 1.4448 - val_accuracy: 0.4400
Epoch 2/6
480/480 [==============================] - 4s 8ms/step - loss: 1.4427 - accuracy: 0.4777 - val_loss: 1.1897 - val_accuracy: 0.5911
Epoch 3/6
480/480 [==============================] - 4s 8ms/step - loss: 1.0287 - accuracy: 0.6446 - val_loss: 1.0876 - val_accuracy: 0.6378
Epoch 4/6
480/480 [==============================] - 4s 8ms/step - loss: 0.7250 - accuracy: 0.7524 - val_loss: 1.1369 - val_accuracy: 0.6311
Epoch 5/6
480/480 [==============================] - 6s 13ms/step - loss: 0.5888 - accuracy: 0.7900 - val_loss: 1.1661 - val_accuracy: 0.6133
Epoch 6/6
480/480 [==============================] - 5s 11ms/step - loss: 0.5372 - accuracy: 0.8033 - val_loss: 1.1582 - val_accuracy: 0.6200


begin training using onecycle policy with max lr of 0.001...
Epoch 1/6
480/480 [=======

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/fb73bdf8e9cb4ff8b3a19cdbc63e8721
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [12]                : (0.2731179893016815, 0.8033341765403748)
COMET INFO:     batch_accuracy [576]         : (0.0625, 1.0)
COMET INFO:     batch_loss [576]             : (0.19524267315864563, 3.96859073638916)
COMET INFO:     epoch_duration [12]          : (3.7189999999973224, 6.437999999994645)
COMET INFO:     loss [12]                    : (0.5321366786956787, 2.0927815437316895)
COMET INFO:     val_accuracy [12]            : (0.4399999976158142, 0.6422222256660461)
COMET INFO:     val_loss [12]                : (1.0875886678695679, 1.4448007345199585)
COMET INFO:     validate_batch_accuracy [



begin training using onecycle policy with max lr of 0.001...
Epoch 1/4
480/480 [==============================] - 6s 10ms/step - loss: 1.8318 - accuracy: 0.3685 - val_loss: 1.2717 - val_accuracy: 0.5756
Epoch 2/4
480/480 [==============================] - 5s 9ms/step - loss: 1.0690 - accuracy: 0.6220 - val_loss: 1.2533 - val_accuracy: 0.5889
Epoch 3/4
480/480 [==============================] - 4s 9ms/step - loss: 0.7162 - accuracy: 0.7479 - val_loss: 1.1885 - val_accuracy: 0.6156
Epoch 4/4
480/480 [==============================] - 4s 9ms/step - loss: 0.5735 - accuracy: 0.7953 - val_loss: 1.1914 - val_accuracy: 0.6178


begin training using onecycle policy with max lr of 0.001...
Epoch 1/4
480/480 [==============================] - 6s 10ms/step - loss: 1.8244 - accuracy: 0.3593 - val_loss: 1.2989 - val_accuracy: 0.5267
Epoch 2/4
480/480 [==============================] - 5s 9ms/step - loss: 1.0610 - accuracy: 0.6196 - val_loss: 1.0760 - val_accuracy: 0.6200
Epoch 3/4
480/480 [=======

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/0618dc74e78149db8a2a0a2b70b0c570
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [8]                 : (0.3593383729457855, 0.7952591776847839)
COMET INFO:     batch_accuracy [384]         : (0.125, 0.875)
COMET INFO:     batch_loss [384]             : (0.45421352982521057, 3.334613084793091)
COMET INFO:     epoch_duration [8]           : (4.327999999994063, 6.312999999994645)
COMET INFO:     loss [8]                     : (0.5734682679176331, 1.8317608833312988)
COMET INFO:     val_accuracy [8]             : (0.5266666412353516, 0.6222222447395325)
COMET INFO:     val_loss [8]                 : (1.0760090351104736, 1.2989141941070557)
COMET INFO:     validate_batch_accuracy 

In [93]:
# numeric input layers
numerical_input = Input(shape=(num_train.shape[1],), name='numerical_input')
x = Dense(
    units=900,
    activation='elu',
    kernel_initializer='HeNormal')(numerical_input)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

x = Dense(
    units=150,
    activation='elu',
    kernel_initializer='HeNormal')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
numerical_output = Flatten()(x) 

# categorical input layers
categorical_input = [time_signature_input,key_input,mode_input]
concatenated = Concatenate(name='category')([time_signature_embedded, key_embedded, mode_embedded])
embedded_input = Flatten()(concatenated)
y = Dense(
    units=250,
    activation='elu',
    kernel_initializer='HeUniform')(embedded_input)
y = Dropout(0.5)(y)
y = BatchNormalization()(y)

y = Dense(
    units=200,
    activation='elu',
    kernel_initializer='HeUniform')(y)
y = Dropout(0.3)(y)
y = BatchNormalization()(y)
cat_out = y

# final layer and output
final_layer = Concatenate()([numerical_output, model2.output, cat_out])
final_layer = Dense(
                units=150,
                activation='elu',
                kernel_initializer='HeNormal')(final_layer)

final_model_output = Dense(8,activation='softmax', kernel_initializer='HeNormal')(final_layer)

model = Model(inputs=[numerical_input] + [model2.input] + [categorical_input],outputs=final_model_output,name='final_output')
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [94]:
learner = ktrain.get_learner(
    model,
    train_data=train_data,
    val_data=valid_data,
    batch_size=16
)
learner.fit_onecycle(
    lr=0.001,
    epochs=2,
    class_weight=class_weight,
)



begin training using onecycle policy with max lr of 0.001...
Epoch 1/2
480/480 [==============================] - 5s 9ms/step - loss: 1.6966 - accuracy: 0.4176 - val_loss: 1.2864 - val_accuracy: 0.5533
Epoch 2/2
480/480 [==============================] - 4s 8ms/step - loss: 0.8963 - accuracy: 0.6889 - val_loss: 1.0585 - val_accuracy: 0.6378


In [95]:
predictor = ktrain.get_predictor(model=learner.model, preproc=preproc2)

In [96]:
predictor.save('audio_meta_text_nbsvm_model')